In [192]:
# =============================================================
# PARTE 1 - CARREGAMENTO DO DATASET E RENOMEAÇÃO DAS COLUNAS
# =============================================================
import pandas as pd  
import numpy as np  
import re  
from datetime import datetime  
import os

INPUT_PATH = "vendas_modificado.csv"
OUTPUT_PATH = "vendas_limpo.csv"

# Cria a pasta de saída se necessário
if os.path.dirname(OUTPUT_PATH):
    os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

# Carregar os dados  
df = pd.read_csv('/kaggle/input/ciencia-de-dados/vendas_modificado(in).csv')  
print("Dimensões iniciais:", df.shape)  
print("\nAmostra dos dados iniciais:")  
print(df.head()) 

Dimensões iniciais: (368752, 17)

Amostra dos dados iniciais:
   id_da_compra        data      hora                      cliente  \
0         13679   3/20/2021  23:35:51            LUCAS Araujo Kuhn   
1         28070  10/30/2020   9:00:53         MICAEL SOUZA RONCETE   
2         47484    6/9/2021  15:30:28    FELIPE AUGUSTO NERY SILVA   
3         20809    6/4/2022   8:41:23        LEVI RIBEIRO AMORIM     
4         47123    5/4/2019  13:38:45  GABRIEL MATOS LIMA DA CUNHA   

            produto     valor  quantidade   total                status  \
0  Queijo Mussarela  R$ 16,87          13  239.31  Pagamento Confirmado   
1   Molho de Tomate   R$ 3,25           3    9.75  Pagamento Confirmado   
2      Água Mineral   R$ 1,63           9   36.67  Pagamento Confirmado   
3            Carvão   R$ 8,74           4   54.96          Em Separação   
4              Café   R$ 9,48           2   18.96          Em Separação   

                  cidade estado    pais        cep  frete  \
0    

In [193]:
# =============================================================
# PARTE 2 - FUNÇÃO DE CARREGAMENTO E ANÁLISE INICIAL DOS DADOS
# =============================================================
def carregar_dados(input_path):
    try:
        df = pd.read_csv(input_path, encoding='utf-8')
        colunas = [
            "ID_DA_COMPRA", "DATA", "HORA", "CLIENTE", "PRODUTO", "VALOR", "QUANTIDADE",
            "TOTAL", "STATUS", "CIDADE", "ESTADO", "PAIS", "CEP", "FRETE", "PAGAMENTO",
            "VENDEDOR", "MARCA"
        ]
        df.columns = colunas

        stats_originais = {
            'total_registros': len(df),
            'colunas': df.columns.tolist(),
            'missing_values': df.isnull().sum().to_dict(),
            'duplicatas': df.duplicated().sum()
        }
        return df, stats_originais
    except Exception as e:
        raise Exception(f"Erro ao carregar arquivo: {str(e)}")

In [194]:
# =============================================================
# PARTE 3 - TRATAMENTO DE VALORES FALTANTES
# =============================================================
def tratar_nulos(df):
    # Conversão para numérico e preenchimento
    num_cols = ['VALOR', 'QUANTIDADE', 'FRETE', 'TOTAL']
    df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')
    df[num_cols] = df[num_cols].fillna({
        'VALOR': 0.0,
        'QUANTIDADE': 1,
        'FRETE': 0.0,
        'TOTAL': 0.0
    })

    # Preenchimento e limpeza de textos
    texto_cols = ['CLIENTE', 'PRODUTO', 'STATUS', 'CIDADE', 'ESTADO',
                  'PAIS', 'PAGAMENTO', 'VENDEDOR', 'MARCA']
    df[texto_cols] = df[texto_cols].fillna('desconhecido').apply(lambda x: x.str.strip())

    # Conversão de datas e horas
    df['DATA'] = pd.to_datetime(df['DATA'], errors='coerce').fillna(pd.Timestamp('2000-01-01'))
    df['HORA'] = pd.to_datetime(df['HORA'], format='%H:%M:%S', errors='coerce')
    df['HORA'] = df['HORA'].dt.strftime('%H:%M:%S').fillna('00:00:00')

    return df

In [195]:
# =============================================================
# PARTE 4 - PADRONIZAÇÃO DE NOMES E FORMATOS
# =============================================================
def padronizar_dados(df):
    # Minúsculas e limpeza de texto
    texto_cols = ['CLIENTE', 'PRODUTO', 'STATUS', 'CIDADE', 'ESTADO',
                  'PAIS', 'PAGAMENTO', 'VENDEDOR', 'MARCA']
    df[texto_cols] = df[texto_cols].apply(lambda x: x.str.lower().str.strip())

    # Mapeamento de padrões de produtos
    produtos_regex = {
        r'.*iphone.*': 'iphone',
        r'.*samsung.*galaxy.*': 'samsung galaxy',
        r'.*tv.*led.*': 'tv led',
        r'.*laptop.*|.*notebook.*': 'notebook',
        r'.*macbook.*': 'macbook',
        r'.*tablet.*': 'tablet'
    }
    for padrao, valor in produtos_regex.items():
        df['PRODUTO'] = df['PRODUTO'].str.replace(padrao, valor, regex=True)

    return df

In [196]:
# =============================================================
# PARTE 5 - TRATAMENTO DO CAMPO CEP
# =============================================================
def tratar_cep(df):
    df['CEP'] = df['CEP'].astype(str)
    df['CEP'] = df['CEP'].str.replace(r'\D', '', regex=True)
    df['CEP'] = df['CEP'].str.pad(8, fillchar='0')
    df.loc[~df['CEP'].str.match(r'^[0-9]{8}$'), 'CEP'] = '00000000'
    return df


In [197]:
# =============================================================
# PARTE 6 - VALIDAÇÃO E CORREÇÃO DE CAMPOS NUMÉRICOS
# =============================================================
def validar_e_corrigir(df):
    df['TOTAL_CALCULADO'] = df['VALOR'] * df['QUANTIDADE'] + df['FRETE']
    df['DIF_TOTAL'] = abs(df['TOTAL'] - df['TOTAL_CALCULADO'])
    df['TOTAL'] = np.where(df['DIF_TOTAL'] > 0.01, df['TOTAL_CALCULADO'], df['TOTAL'])
    df.drop(columns=['TOTAL_CALCULADO', 'DIF_TOTAL'], inplace=True)

    df = df[(df['QUANTIDADE'] > 0) & (df['QUANTIDADE'] <= 1000)]
    df = df[(df['VALOR'] >= 0) & (df['VALOR'] <= 100000)]
    df = df[(df['FRETE'] >= 0) & (df['FRETE'] <= 5000)]

    return df

In [198]:
# =============================================================
# PARTE 7 - VALIDAÇÃO DE ESTADOS BRASILEIROS
# =============================================================
def validar_estados(df):
    estados_validos = [
        'ac', 'al', 'ap', 'am', 'ba', 'ce', 'df', 'es', 'go', 'ma',
        'mt', 'ms', 'mg', 'pa', 'pb', 'pr', 'pe', 'pi', 'rj', 'rn',
        'rs', 'ro', 'rr', 'sc', 'sp', 'se', 'to', 'desconhecido'
    ]
    return df[df['ESTADO'].isin(estados_validos)]

In [199]:
# =============================================================
# PARTE 8 - REMOÇÃO DE DUPLICATAS E AJUSTE DE ÍNDICES
# =============================================================
def remover_duplicatas(df):
    df = df.drop_duplicates()
    return df.reset_index(drop=True)

In [200]:
# =============================================================
# PARTE 9 - EXPORTAÇÃO DOS DADOS E GERAÇÃO DE RELATÓRIO
# =============================================================
def exportar_resultado(df, output_path, stats_originais):
    try:
        df.to_csv(output_path, index=False, encoding='utf-8')
        removidos = stats_originais['total_registros'] - len(df)
        percentual = (len(df) / stats_originais['total_registros']) * 100

        relatorio = f"""
        ===============================
        RELATÓRIO DE LIMPEZA DE DADOS
        ===============================

        DADOS ORIGINAIS:
        - Total de registros: {stats_originais['total_registros']}
        - Colunas: {', '.join(stats_originais['colunas'])}
        - Valores faltantes: {sum(stats_originais['missing_values'].values())}
        - Registros duplicados: {stats_originais['duplicatas']}

        DADOS FINAIS:
        - Registros válidos: {len(df)}
        - Removidos: {removidos}
        - Percentual mantido: {percentual:.2f}%

        Arquivo salvo em: {output_path}
        Data de execução: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        """
        print(relatorio)
    except Exception as e:
        raise Exception(f"Erro ao salvar arquivo limpo: {str(e)}")


In [201]:
# Exibição final e salvamento do arquivo  
print("\nAmostra final dos dados limpos:")  
print(df.head())  
  
# Salvar arquivo final  
df.to_csv('vendas_modificado_final_completo.csv', index=False)  
print("\nArquivo final salvo como: vendas_modificado_final_completo.csv") 


Amostra final dos dados limpos:
   id_da_compra        data      hora                      cliente  \
0         13679   3/20/2021  23:35:51            LUCAS Araujo Kuhn   
1         28070  10/30/2020   9:00:53         MICAEL SOUZA RONCETE   
2         47484    6/9/2021  15:30:28    FELIPE AUGUSTO NERY SILVA   
3         20809    6/4/2022   8:41:23        LEVI RIBEIRO AMORIM     
4         47123    5/4/2019  13:38:45  GABRIEL MATOS LIMA DA CUNHA   

            produto     valor  quantidade   total                status  \
0  Queijo Mussarela  R$ 16,87          13  239.31  Pagamento Confirmado   
1   Molho de Tomate   R$ 3,25           3    9.75  Pagamento Confirmado   
2      Água Mineral   R$ 1,63           9   36.67  Pagamento Confirmado   
3            Carvão   R$ 8,74           4   54.96          Em Separação   
4              Café   R$ 9,48           2   18.96          Em Separação   

                  cidade estado    pais        cep  frete  \
0                Niterói     RJ  B